<a href="https://colab.research.google.com/github/uday1277/red-wine/blob/main/Copy_of_wine_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install data

  Preparing metadata (setup.py) ... done
  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7227 sha256=ae0d0530c1e5def7d7657461c256ddf0890cc42655350d5e48c4fca4d74d86d5
  Stored in directory: /root/.cache/pip/wheels/d0/e8/fa/e253c256048ea58d99a8abb5e751abb6a838af6f12887b5418
Successfully built data


In [ ]:
!pip install MessagePassing

ERROR: Could not find a version that satisfies the requirement MessagePassing (from versions: none)
ERROR: No matching distribution found for MessagePassing


In [ ]:
!pip install add_self_loops

ERROR: Could not find a version that satisfies the requirement add_self_loops (from versions: none)
ERROR: No matching distribution found for add_self_loops


In [ ]:
!pip install torch_geometric.nn

ERROR: Could not find a version that satisfies the requirement torch_geometric.nn (from versions: none)
ERROR: No matching distribution found for torch_geometric.nn


In [ ]:
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install torch


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Try to read the CSV file
try:
    df = pd.read_csv("dataset.csv", encoding='latin1')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    # Handle the error or exit the script

# Assuming 'quality' is the target variable
# Map target variable to start from 0
y_mapping = {label: idx for idx, label in enumerate(sorted(df['quality'].unique()))}
df['quality'] = df['quality'].map(y_mapping)

# Split the data into features (X) and target variable (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a Decision Tree model
model = DecisionTreeClassifier(random_state=7)

# Fit the model to the training data
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Reverse map predicted values to original labels
y_pred_original = pd.Series(y_pred).map({v: k for k, v in y_mapping.items()})

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy Percentage: {accuracy * 100}\n\n\n")


Error reading CSV file: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3

Accuracy Percentage: 59.166666666666664





In [ ]:
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import ChebConv
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Try to read the CSV file
try:
    df = pd.read_csv("dataset.csv", encoding='latin1')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    # Handle the error or exit the script

# Assuming 'quality' is the target variable
# Map target variable to start from 0
y_mapping = {label: idx for idx, label in enumerate(sorted(df['quality'].unique()))}
df['quality'] = df['quality'].map(y_mapping)

# Split the data into features (X) and target variable (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
x_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float)

# Create a graph data structure using PyTorch Geometric
num_nodes = X_train_scaled.shape[0]
edge_index = torch.tensor([[i, i] for i in range(num_nodes)], dtype=torch.long).t().contiguous()

# Define an improved Graph Convolutional Network (GCN) model
class ImprovedGNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels1, hidden_channels2, out_channels, dropout_rate=0.5):
        super(ImprovedGNNModel, self).__init__()
        self.conv1 = ChebConv(in_channels, hidden_channels1, K=2)
        self.conv2 = ChebConv(hidden_channels1, hidden_channels2, K=2)
        self.conv3 = ChebConv(hidden_channels2, out_channels, K=2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.dropout(x)
        x = F.relu(self.conv3(x, edge_index))
        return F.log_softmax(x, dim=1)

# Initialize and train the improved GNN model
improved_model = ImprovedGNNModel(
    in_channels=X_train_scaled.shape[1],
    hidden_channels1=64,
    hidden_channels2=32,
    out_channels=len(y_mapping),
    dropout_rate=0.5
)

optimizer = torch.optim.Adam(improved_model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Convert data to PyTorch Geometric Data object
data = Data(x=x_train_tensor, edge_index=edge_index, y=y_train_tensor)
loader = DataLoader([data], batch_size=1, shuffle=True)

# Training loop
for epoch in range(200):
    improved_model.train()
    for batch in loader:
        optimizer.zero_grad()
        out = improved_model(batch)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

# Evaluate the improved GNN model on the test set
improved_model.eval()
with torch.no_grad():
    test_data = Data(x=x_test_tensor, edge_index=edge_index)
    out = improved_model(test_data)
    y_pred = out.argmax(dim=1).numpy()

# Reverse map predicted values to original labels
y_pred_original = pd.Series(y_pred).map({v: k for k, v in y_mapping.items()})

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy Percentage: {accuracy * 100}\n\n\n")


Error reading CSV file: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3



/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Accuracy Percentage: 61.04166666666667





In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Try to read the CSV file
try:
    df = pd.read_csv("dataset.csv", encoding='latin1')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    # Handle the error or exit the script

# Assuming 'quality' is the target variable
# Map target variable to start from 0
y_mapping = {label: idx for idx, label in enumerate(sorted(df['quality'].unique()))}
df['quality'] = df['quality'].map(y_mapping)

# Split the data into features (X) and target variable (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)

# Fit the model to the training data
knn_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = knn_model.predict(X_test_scaled)

# Reverse map predicted values to original labels
y_pred_original = pd.Series(y_pred).map({v: k for k, v in y_mapping.items()})

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy Percentage: {accuracy * 100}\n\n\n")


Error reading CSV file: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3

Accuracy Percentage: 58.75





In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Try to read the CSV file
try:
    df = pd.read_csv("dataset.csv", encoding='latin1')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    # Handle the error or exit the script

# Assuming 'quality' is the target variable
# Map target variable to start from 0
y_mapping = {label: idx for idx, label in enumerate(sorted(df['quality'].unique()))}
df['quality'] = df['quality'].map(y_mapping)

# Split the data into features (X) and target variable (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an SVM model
svm_model = SVC(kernel='rbf', C=1.0, random_state=7)

# Fit the model to the training data
svm_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_scaled)

# Reverse map predicted values to original labels
y_pred_original = pd.Series(y_pred).map({v: k for k, v in y_mapping.items()})

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy Percentage: {accuracy * 100}\n\n\n")


Error reading CSV file: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3

Accuracy Percentage: 61.04166666666667





In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Try to read the CSV file
try:
    df = pd.read_csv("dataset.csv", encoding='latin1')
    print("CSV file read successfully.")
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    # Handle the error or exit the script

# Assuming 'quality' is the target variable
# Map target variable to start from 0
y_mapping = {label: idx for idx, label in enumerate(sorted(df['quality'].unique()))}
df['quality'] = df['quality'].map(y_mapping)

# Split the data into features (X) and target variable (y)
X = df.drop('quality', axis=1)
y = df['quality']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a Logistic Regression model
logistic_model = LogisticRegression(random_state=7)

# Fit the model to the training data
logistic_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = logistic_model.predict(X_test_scaled)

# Reverse map predicted values to original labels
y_pred_original = pd.Series(y_pred).map({v: k for k, v in y_mapping.items()})

# Print accuracy
accuracy = accuracy_score(y_test, y_pred_original)
print(f"Accuracy Percentage: {accuracy * 100}\n\n\n")


Error reading CSV file: Error tokenizing data. C error: Expected 1 fields in line 4, saw 3

Accuracy Percentage: 57.70833333333333



